<a href="https://colab.research.google.com/github/aaabhijith13/linkedIN_posts/blob/main/spark_streaming_realtime_analytics_detailed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Spark Streaming: Real-Time Analytics with Live API Data

This notebook demonstrates **Spark Streaming using DStreams** to perform
**near real-time analytics** on continuously arriving data.

The goal is not just to ingest data, but to **extract real-time insights**
such as trends, rolling averages, and anomaly signals.

---

## What This Notebook Covers

- Live data ingestion from an external API
- Micro-batch processing model
- DStreams and RDD execution
- Window-based analytics
- Trend detection logic
- Fault tolerance concepts
- Practical limitations and best practices

---

## Use Case

**Real-time cryptocurrency market analytics**:
- Track live prices
- Compute rolling averages
- Detect short-term trends
- Identify sudden price movements

This pattern applies equally to:
- Live transcripts
- Sensor data
- Clickstreams
- Financial tick data



## 1. Spark Streaming Setup

Spark Streaming operates on top of Spark using a **micro-batch model**.
Each batch is processed as a standard Spark job.

We create:
- `SparkContext`
- `StreamingContext`

Batch interval: **10 seconds**


In [1]:
!pip install pyspark

!apt-get install openjdk-8-jdk-headless -qq


In [2]:
# Source - https://stackoverflow.com/a
# Posted by Anup Ash, modified by community. See post 'Timeline' for change history
# Retrieved 2026-01-13, License - CC BY-SA 3.0

!export PYSPARK_SUBMIT_ARGS="--master local[2] pyspark-shell"


In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext("local[2]", appName="SparkStreamingRealTimeAnalytics")
ssc = StreamingContext(sc, batchDuration=10)

ssc.checkpoint("/tmp/spark_streaming_checkpoint")

/usr/local/lib/python3.12/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(



## 2. Why API-Based Streaming Needs Polling

Spark Streaming does not directly consume REST APIs.
Instead, we:
1. Poll the API at fixed intervals
2. Convert responses into RDDs
3. Feed them into Spark using `queueStream`

In production systems:
- APIs → Kafka → Spark Streaming


In [4]:

import requests
import time
from datetime import datetime



## 3. Fetching Live Data from an External API

We use the **CoinGecko API** to fetch live Bitcoin prices.
Each API response is timestamped to support time-based analytics.


In [5]:
import requests
from datetime import datetime, timezone

SESSION = requests.Session()
SESSION.headers.update({
    "User-Agent": "spark-streaming-demo/1.0"
})

def fetch_crypto_price():
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {"ids": "bitcoin", "vs_currencies": "usd"}

    resp = SESSION.get(url, params=params, timeout=10)

    # Fail fast on HTTP errors (429, 403, 5xx, etc.)
    if resp.status_code != 200:
        # Return None so streaming can continue
        return None, f"HTTP {resp.status_code}: {resp.text[:200]}"

    # Defensive JSON parsing
    try:
        payload = resp.json()
    except Exception as e:
        return None, f"JSON decode error: {e}; body={resp.text[:200]}"

    # Validate expected shape
    btc = payload.get("bitcoin")
    if not isinstance(btc, dict) or "usd" not in btc:
        return None, f"Unexpected payload: {str(payload)[:200]}"

    price = btc["usd"]
    ts = datetime.now(timezone.utc).isoformat()
    return (ts, float(price)), None



## 4. Creating a DStream from Polled API Data

Each API poll result becomes an RDD.
Spark Streaming treats this as a continuous data stream.


In [6]:

rdd_queue = [sc.emptyRDD()]
raw_stream = ssc.queueStream(rdd_queue, oneAtATime=True)

price_stream = (
    raw_stream
    .filter(lambda x: x[0] is not None)   # keep successful records
    .map(lambda x: x[0])                  # UNWRAP HERE
)




## 5. Basic Transformations

We extract the price value and prepare the data for analytics.


In [7]:
# Extract price
prices = price_stream.map(lambda x: x[1])

# Pair for aggregation
price_pairs = prices.map(lambda p: ("BTC", p))


## 6. Per-Batch Analytics

Each micro-batch represents a near real-time snapshot.


In [8]:

# Compute min, max, and average per batch
batch_stats = price_pairs.groupByKey().mapValues(
    lambda ps: {
        "min": min(ps),
        "max": max(ps),
        "avg": sum(ps) / len(ps)
    }
)



## 7. Window-Based Analytics (Rolling Insights)

Window operations allow analytics across **multiple batches**.

Window configuration:
- Window length: 60 seconds
- Slide interval: 20 seconds


In [9]:

windowed_avg = price_pairs.groupByKeyAndWindow(
    windowDuration=60,
    slideDuration=20
).mapValues(lambda ps: sum(ps) / len(ps))


In [10]:
batch_stats.pprint()
windowed_avg.pprint()


## 8. Trend Detection Logic

We detect short-term trends by comparing:
- Current window average
- Previous window average

This is a simplified trend signal.


In [11]:

def detect_trend(rdd):
    data = rdd.collect()
    if len(data) >= 2:
        prev = data[-2][1]
        curr = data[-1][1]
        if curr > prev:
            print("📈 Upward trend detected")
        elif curr < prev:
            print("📉 Downward trend detected")
        else:
            print("➡️ Stable trend")

windowed_avg.foreachRDD(detect_trend)



## 9. Fault Tolerance and Resilience

Spark Streaming is fault tolerant because:
- Each batch is an RDD
- RDD lineage enables recomputation
- Checkpointing preserves stateful operations

If a worker node fails:
- Lost partitions are recomputed automatically



## 10. Starting the Streaming Job

The loop below continuously:
- Polls the API
- Feeds data into Spark
- Triggers micro-batch execution


In [12]:

ssc.start()

try:
    while True:
        record = fetch_crypto_price()
        print(record)
        if record:
            rdd_queue.append(sc.parallelize([record]))
        time.sleep(10)
except KeyboardInterrupt:
    pass

ssc.stop(stopSparkContext=True, stopGraceFully=True)


(('2026-01-14T03:08:56.332727+00:00', 95300.0), None)
-------------------------------------------
Time: 2026-01-14 03:09:00
-------------------------------------------

(('2026-01-14T03:09:06.510957+00:00', 95300.0), None)
-------------------------------------------
Time: 2026-01-14 03:09:10
-------------------------------------------

-------------------------------------------
Time: 2026-01-14 03:09:10
-------------------------------------------

-------------------------------------------
Time: 2026-01-14 03:09:20
-------------------------------------------

-------------------------------------------
Time: 2026-01-14 03:09:30
-------------------------------------------

-------------------------------------------
Time: 2026-01-14 03:09:30
-------------------------------------------




## Key Takeaways

- Spark Streaming uses **micro-batches**
- DStreams are sequences of RDDs
- Window operations enable real-time analytics
- In-memory processing reduces latency
- Spark Streaming is **near real-time**, not event-by-event

---

## Next Steps

- Kafka + Spark Streaming integration
- Structured Streaming (modern replacement for DStreams)
- Real-time ML inference pipelines
- Stateful stream processing

This notebook serves as a **foundation for real-world streaming analytics**.
